>GUI #1 (5pts)
>* Using the quotes API, build a GUI that provides a drop-down menu of people randomly chosen from the API
>* I want a user to be able to pick from that list and receive a quote that corresponds to that person 
>* Ideally, I want to be able to reset that menu & allow people to choose from a new list 


In [1]:
import requests
import tkinter as tk
import random

def get_quote_authors(num_authors):
    # The API does not support randomly choosing authors
    # Instead, we request random quotes and take their authors
    quotes_per_request = num_authors*2
    num_requests = 0
    all_authors = set()
    url = "https://api.quotable.io/quotes/random"
    query = f"?limit={quotes_per_request}"
    # Continue to make requests if we do not have enough authors
    while len(all_authors) < num_authors:
        # If we make too many requests, clearly something is wrong, so abort
        if num_requests > 5:
            print("Too many requests! Something went wrong...")
            break
        quotes_request = requests.get(url + query).json()
        num_requests += 1
        for quote in quotes_request:
            # Terminate the loop mid-way once we have enough authors
            if len(all_authors) >= num_authors:
                break
            author =  quote["author"]
            all_authors.add(author)
    return all_authors

def get_quote(author):
    url = "https://api.quotable.io/quotes/random"
    query = f"?author={author}"
    quote_request = requests.get(url + query).json()
    quote = quote_request[0]["content"]
    return quote

def update_gui_quote():
    global root
    global current_author
    global quote_text
    author_name = current_author.get()
    new_quote = get_quote(author_name)
    new_text = f"\"{new_quote}\" ~ {author_name}"
    quote_text.configure(state="normal")
    quote_text.delete(0.0, tk.END)
    quote_text.insert(tk.END, new_text)
    quote_text.configure(state="disabled")

def reset_authors():
    global authors_dropdown
    global current_author
    num_authors = 5
    all_authors = get_quote_authors(num_authors)
    authors_dropdown["menu"].delete(0, tk.END)
    for author in all_authors:
        # Esoteric syntax for adding new options to the menu
        # https://stackoverflow.com/questions/17580218/changing-the-options-of-a-optionmenu-when-clicking-a-button
        authors_dropdown["menu"].add_command(label=author, command=tk._setit(current_author, author))
    current_author.set("Select an author")
        
root = tk.Tk()
root.title("Random historical quotes generator")

heading = tk.Label(root, text="Historical Quotes Generator")

dropdown_frame = tk.Frame(root)

current_author = tk.StringVar()
authors_dropdown = tk.OptionMenu(dropdown_frame, current_author, "placeholder_option")
reset_authors()

reset_authors_button = tk.Button(dropdown_frame, text="Get new authors", command=reset_authors)

quote_text = tk.Text(root, font=("Georgia", 10), height=4, width=40, padx=30, pady=50)
quote_text.insert(tk.END, "Please select an author from the dropdown to get a quote")
quote_text.configure(state="disabled")

new_quote_button = tk.Button(root, text="Press me to get a quote", command=update_gui_quote)

heading.pack()
dropdown_frame.pack()
authors_dropdown.pack(side=tk.LEFT)
reset_authors_button.pack(side=tk.RIGHT)
quote_text.pack()
new_quote_button.pack()
root.mainloop()

> GUI #2 (5pts)
> * Go back to the weather API we used a few weeks ago and build a GUI that displays the 7 day forecast. Model it off of the weather forecaster you typically use (like an app or weather.com) 
> * Try using one widget we haven’t yet used in class – identify it with a comment  

In [2]:
import requests
import tkinter as tk

def get_weather_forecast(lat, lon):
    metadata_url = f"https://api.weather.gov/points/{lat},{lon}"
    metadata_request = requests.get(metadata_url).json()
    forecast_url = metadata_request["properties"]["forecast"]
    forecast_request = requests.get(forecast_url).json()
    print(forecast_url)
    weather_periods = forecast_request["properties"]["periods"]
    return weather_periods

def check_button_toggle():
    global check_button
    check_button.config(text="Nice")
    
# Binghamton has coordinates 42.0894° N, 75.9695° W
lat = 42.0987
lon = -75.9180

bing_forecast = get_weather_forecast(lat, lon)
print(bing_forecast)
forecast_days = []
start_index = 0
end_index = len(bing_forecast)

# If the first subperiod is "Tonight", then there is only one entry for "the current day"
# This breaks the code, as it expects two entries for any given day (the first half of the day, and the last half of the day)
# Therefore, we handle "today" manually and shift the indexes to deal with the rest of the days as normal
if bing_forecast[0]["name"] == "Tonight":
    start_index = 1
    end_index -= 1
    lower_period = bing_forecast[0]
    lower_period = {"name": lower_period["name"],
                    "startTime": lower_period["startTime"],
                    "temperature": lower_period["temperature"], 
                    "detailedForecast": lower_period["detailedForecast"]}
    dummy_period = {"name": "Today",
                   "startTime": lower_period["startTime"],
                   "temperature": "N/A",
                   "detailedForecast": "No data for the given period."}
    forecast_days.append((dummy_period, lower_period))

# We format our list as [(Today, Today Night), (Next Day, Next Day Night), ...]
for period_index in range(start_index, end_index, 2):
    upper_period = bing_forecast[period_index]
    upper_period = {"name": upper_period["name"],
                    "startTime": upper_period["startTime"],
                    "temperature": upper_period["temperature"], 
                    "detailedForecast": upper_period["detailedForecast"]}
    lower_period = bing_forecast[period_index+1]
    lower_period = {"name": lower_period["name"],
                    "startTime": upper_period["startTime"],
                    "temperature": lower_period["temperature"], 
                    "detailedForecast": lower_period["detailedForecast"]}
    forecast_days.append((upper_period, lower_period))

root = tk.Tk()
root.title("Binghamton Seven Day Forecast")

heading = tk.Label(root, text="Binghamton Seven Day Forecast")

# This is the "widget we haven’t yet used in class"
# Its only purpose is to be checked, so you can validate that you are having a good day
check_button = tk.Checkbutton(root, text="Check this button if you're having a good day", command=check_button_toggle)
forecast_frame = tk.Frame(root)

for column, period in enumerate(forecast_days):
    for row, subperiod in enumerate(period):
        subperiod_frame = tk.Frame(forecast_frame)
        date = subperiod["startTime"][0:10]
        date = tk.Label(subperiod_frame, text=date)
        name = tk.Label(subperiod_frame, text=subperiod["name"])
        temp = str(subperiod["temperature"]) + " F"
        temp = tk.Label(subperiod_frame, text=temp)
        forecast = tk.Text(subperiod_frame, font=("Georgia", 10), height=10, width=20)
        forecast.insert(tk.END, subperiod["detailedForecast"])
        
        if row == 0:
            date.grid(row=0, column=column)
        name.grid(row=1,column=column)
        temp.grid(row=2,column=column)
        forecast.grid(row=3,column=column)
            
        subperiod_frame.grid(row=row, column=column)

heading.pack()
check_button.pack()
forecast_frame.pack()
root.mainloop()